In [1]:
import pandas as pd
import json
import numpy as np
import datetime
import os
from haversine import haversine
os.getcwd()

zip_centers=json.load(open("/home/jian/Docs/Geo_mapping/updated_zip_centers_JL_2019-05-23.json","r"))

In [2]:
# Email from Bruce 20190523 4:24 pm

'''
Detroit
Greensboro HP WS
Hartford & New Haven
Jacksonville
Kansas City
Oklahoma City
Portland
'''

zip_DMA=pd.read_excel("/home/jian/Docs/Geo_mapping/Zips by DMA by County16-17 nielsen.xlsx",dtype=str,skiprows=1,usecols=[0,2])
zip_DMA.columns=['zip_cd','DMA']
zip_DMA=zip_DMA.drop_duplicates()

In [3]:
# print(sorted(zip_DMA['DMA'].unique().tolist()))

DMA_7_list=["DETROIT","GREENSBORO-H.POINT-W.SALEM","HARTFORD & NEW HAVEN","JACKSONVILLE","KANSAS CITY","OKLAHOMA CITY","PORTLAND, OR"]
print(len(DMA_7_list))

zip_DMA_7_Mkts=zip_DMA[zip_DMA['DMA'].isin(DMA_7_list)]
print(zip_DMA_7_Mkts['DMA'].nunique())

7
7


In [4]:
# 20190518, 4 weeks back
most_recent_sales=pd.read_csv("/home/jian/BiglotsCode/outputs/combined_sales_long_2019-05-18.csv",
                              dtype=str,usecols=['location_id','sales','week_end_date'])
most_recent_sales['sales']=most_recent_sales['sales'].astype(float)

most_recent_sales_weeks=sorted(most_recent_sales['week_end_date'].unique().tolist(),reverse=True)
last_4_weeks=most_recent_sales_weeks[:4]

most_recent_sales=most_recent_sales[most_recent_sales['week_end_date'].isin(last_4_weeks)]
most_recent_sales=most_recent_sales[most_recent_sales['sales']>0]

most_recent_sales_0518=most_recent_sales[most_recent_sales['week_end_date']=="2019-05-18"]
most_recent_sales_0518=most_recent_sales_0518[['location_id','sales']].rename(columns={"sales":"sales_2019-05-18"})
most_recent_sales_count=most_recent_sales.groupby("location_id")['week_end_date'].nunique().to_frame().reset_index().rename(columns={"week_end_date":"4_weeks_count"})
most_recent_sales=pd.merge(most_recent_sales_0518,most_recent_sales_count,on="location_id",how="outer")
most_recent_sales=most_recent_sales.fillna(0)

stores_included=most_recent_sales[(most_recent_sales['4_weeks_count']>=3) | (most_recent_sales['sales_2019-05-18']>0)]['location_id'].tolist()
print(len(stores_included))

1401


In [5]:
store_list_SFTP=pd.read_csv("/home/jian/BigLots/static_files/Store_list/MediaStormStores20190501-135143-036.txt",
                            dtype=str,sep="|",usecols=['location_id','zip_cd','latitude_meas','longitude_meas'])
store_list_SFTP['latitude_meas']=store_list_SFTP['latitude_meas'].astype(float)
store_list_SFTP['longitude_meas']=store_list_SFTP['longitude_meas'].astype(float)

store_list_SFTP['zip_cd']=store_list_SFTP['zip_cd'].apply(lambda x: x.split("-")[0].zfill(5))
store_list_SFTP_included=store_list_SFTP[store_list_SFTP['location_id'].isin(stores_included)]
print(store_list_SFTP_included.shape)

print("Manually google based on the planned stores: ", [x for x in stores_included if x not in store_list_SFTP_included['location_id'].tolist()])

(1400, 4)
Manually google based on the planned stores:  ['5391']


In [6]:
# 5391

# 5391 	1251 BURKEMONT AVENUE	MAGNOLIA PLAZA	MORGANTON	NC	28655
# 1251 Burkemont Ave, Morganton, NC 28655
# Confirmed with the zip
store_list_SFTP_included=store_list_SFTP_included.append(pd.DataFrame({"location_id":"5391","zip_cd":"28655"},index=[0]))
store_list_SFTP_included=store_list_SFTP_included.reset_index()
del store_list_SFTP_included['index']
print(store_list_SFTP_included.shape)
print(store_list_SFTP_included['location_id'].nunique())

(1401, 4)
1401


In [7]:
store_list_SFTP_included=pd.merge(store_list_SFTP_included,zip_DMA_7_Mkts,on="zip_cd",how="left")
store_list_SFTP_included=store_list_SFTP_included[pd.notnull(store_list_SFTP_included['DMA'])]
print(store_list_SFTP_included.shape)
print(store_list_SFTP_included['location_id'].nunique())

store_list_SFTP_included=store_list_SFTP_included.rename(columns={"zip_cd":"store_zip"})

(87, 5)
87


In [8]:
zips_labeled_by_store=pd.read_csv("/home/jian/Projects/Big_Lots/New_TA/zips_in_new_ta/sales_by_zip (Store level).csv",
                                  dtype=str,usecols=['location_id','zip','revenue_flag'])
zips_labeled_by_store=zips_labeled_by_store[zips_labeled_by_store['revenue_flag']!="T"]
store_list_SFTP_included=pd.merge(store_list_SFTP_included,zips_labeled_by_store,on="location_id",how="left")


store_list_SFTP_included_old=store_list_SFTP_included[pd.notnull(store_list_SFTP_included['zip'])]
store_list_SFTP_included_new=store_list_SFTP_included[pd.isnull(store_list_SFTP_included['zip'])]

print(store_list_SFTP_included_old['location_id'].nunique())
print(store_list_SFTP_included_new.shape)

82
(5, 7)


In [9]:
del store_list_SFTP_included_new['zip']
del store_list_SFTP_included_new['revenue_flag']

store_list_SFTP_included_new=store_list_SFTP_included_new.reset_index()
del store_list_SFTP_included_new['index']

all_new_stores_df=pd.DataFrame()
for index,row in store_list_SFTP_included_new.iterrows():
    location_id=row['location_id']
    store_center=[row['latitude_meas'],row['longitude_meas']]
    store_zip=row['store_zip']
    df_store=pd.DataFrame({"location_id":location_id,"zip":store_zip,"revenue_flag":"zip_in_10","dist_miles":np.nan},index=[0])
    for zip_cd in zip_centers.keys():
        dist=haversine(zip_centers[zip_cd],store_center,miles=True)
        
        if dist<=10:
            df=pd.DataFrame({"location_id":location_id,"zip":zip_cd,"revenue_flag":"zip_in_10","dist_miles":dist},index=[0])
            df_store=df_store.append(df)
    df_store=df_store[pd.notnull(df_store['dist_miles'])].append(df_store[pd.isnull(df_store['dist_miles'])])
    df_store=df_store.drop_duplicates("zip")
    all_new_stores_df=all_new_stores_df.append(df_store)
store_list_SFTP_included_new=pd.merge(store_list_SFTP_included_new,all_new_stores_df,on="location_id",how="left")

In [10]:
store_list_SFTP_included_old['dist_miles']=np.nan
for index,row in store_list_SFTP_included_old.iterrows():
    store_center=[row['latitude_meas'],row['longitude_meas']]
    zip_center=zip_centers[row['zip']]
    dist=haversine(store_center,zip_center,miles=True)
    store_list_SFTP_included_old.loc[index,"dist_miles"]=dist

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [11]:
cols_list=['location_id','DMA','store_zip','latitude_meas','longitude_meas','zip','revenue_flag','dist_miles']

store_list_SFTP_included_old=store_list_SFTP_included_old[(store_list_SFTP_included_old['revenue_flag']=="P") | (store_list_SFTP_included_old['dist_miles']<20)]
store_list_SFTP_included_old=store_list_SFTP_included_old[cols_list]
store_list_SFTP_included_new=store_list_SFTP_included_new[cols_list]

output_all=store_list_SFTP_included_old.append(store_list_SFTP_included_new)
output_all=output_all.sort_values(["DMA","location_id","revenue_flag","dist_miles"])

In [12]:
os.getcwd()

'/home/jian/Projects/Big_Lots/Analysis/2019_Q2/Planner_Requests/Zips_7_Markets_Bruce_20190524'

In [15]:
writer=pd.ExcelWriter("/home/jian/Projects/Big_Lots/Analysis/2019_Q2/Planner_Requests/Zips_7_Markets_Bruce_20190524/BL_zips_7_markets_for circ_test_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")



In [16]:
unique_zips_by_DMA=pd.DataFrame()
for DMA,group in output_all.groupby("DMA"):
    group.to_excel(writer,DMA,index=False)
    
    dedup_df=group[['DMA','zip']].drop_duplicates()
    unique_zips_by_DMA=unique_zips_by_DMA.append(dedup_df)
    
unique_zips_by_DMA.to_excel(writer,"unique_per_DMA",index=False)

all_unique_zips=unique_zips_by_DMA[['zip']].drop_duplicates()
all_unique_zips.to_excel(writer,"all_unique_zips",index=False)
writer.save()